# Predicting Financial Performance With Hierarchical Time Series

## Can we use heirarchical time series to capture the structure of companies and industries and predict financial performance?

In [31]:
import calcbench as cb
from hts import HTSRegressor
import hts

In [72]:
accounts = [
    "Revenue",
    "CostOfRevenue",
    "GrossProfit",
    "SGAExpense",
    "OperatingExpenses",
    "InterestExpense",
    "IncomeTaxes",
]

In [73]:
api_data = cb.standardized_data(
    company_identifiers=["msft", "orcl", "ko"],
    metrics=[
        *accounts,
        "SIC_Code",
    ],
    all_history=True,
    period_type="annual",
)

In [36]:
one_digit_SIC_codes = pd.to_numeric(
    api_data["SIC_Code"].iloc[0] // 1000, downcast="integer"
).astype(str)
one_digit_SIC_codes.name = "SIC_Code"

d = api_data.drop(columns="SIC_Code", level="metric")

d = d.drop(d.index[0])  # drop the 0000 column left over from the SIC_Code

d = d.stack().join(one_digit_SIC_codes).set_index("SIC_Code", append=True)

d.index = d.index.set_levels(d.index.levels[0].to_timestamp(), level=0)

d = d.reset_index()

In [37]:
for negative_account in [
    "CostOfRevenue",
    "IncomeTaxes",
    "InterestExpense",
    "OperatingExpenses",
    "SGAExpense",
]:
    d[negative_account] = d[negative_account] * -1

In [38]:
d = pd.melt(d, id_vars=["period", "ticker", "SIC_Code"])

In [39]:
d = d.rename(columns={"variable": "account"})

In [40]:
ht, _, _, = hts.functions.get_hierarchichal_df(
    d,
    level_names=["SIC_Code", "ticker", "account"],
    date_colname="period",
    val_colname="value",
    hierarchy=[["SIC_Code"], ["ticker"], ["account"]],
)

In [30]:
heirarchy = {
    "total": ["7", "2"],
    "7": ["7_MSFT", "7_ORCL"],
    "2": ["2_KO"],
    "7_MSFT": [],
}

In [75]:
tickers_with_SIC_groups = [("KO", 2), ("MSFT", 7), ("ORCL", 7)]
unique_SIC_codes = list(set([str(sic_code) for _, sic_code in tickers_with_SIC_groups]))
heirarchy = {"total": unique_SIC_codes}
for SIC_code in unique_SIC_codes:
    heirarchy[SIC_code] = [
        f"{SIC_code}_{ticker}"
        for ticker, ticker_SIC_code in tickers_with_SIC_groups
        if str(ticker_SIC_code) == str(SIC_code)
    ]
for ticker, SIC_group in tickers_with_sic_groups:
    key = f"{SIC_group}_{ticker}"
    heirarchy[key] = [f"{key}_{account}" for account in accounts]

In [77]:
clf = HTSRegressor(model="prophet", revision_method="OLS", n_jobs=0)

In [ ]:
model = clf.fit(ht, heirarchy)

In [ ]:
predicted = model.predict(steps_ahead=4)

## Predicted "totals" for companies is net-income